# 최단 결로 알고리즘

가장 짧은 경로를 찾는 알고리즘.
DP와 greedy 알고리즘의 한 유형으로 볼 수 있음.

## 다익스트라 최단 경로 알고리즘

Dijkstra... 신기하네

그래프에 여러 개의 노드가 있을 때, 특정한 노드에서 출발하여 다른 노드로 가는 각각의 최단 경로를 구함.
음의 간선이 없을 떄 정상적으로 동작함. greedy 알고리즘으로 분류함.

<br>

**과정**

1. 출발 노드를 정함.
2. 최단 거리 테이블을 초기화함.
3. 방문하지 않은 노드 중 최단 거리가 가장 짧은 노드를 선택함.
4. 해당 노드를 거쳐 다른 노드로 가는 비용을 계산하여 최단 거리 테이블을 갱신함.
5. `3~4` 반복


In [1]:
import sys

In [12]:
inputData = sys.stdin.readline() # input 더 빨리 됨.
INF = int(1e9) # 무한을 의미하는 값, 10억

n, m = map(int, input().split()) # 노드의 개수, 간선의 개수 입력
start = int(input())
graph = [[] for _ in range(n+1)]
visited = [False] * (n+1)
distance = [INF] * (n+1)

for _ in range(m):
    startNode, endNode, cost = map(int, input().split())
    graph[startNode].append((endNode, cost))

### 간단한 다익스트라 알고리즘

알고리즘을 그대로 구현함. 시간복잡도는 $O(V^2)$ $(V: 노드의 개수)$를 가짐.

In [1]:
def getSmallestNode():
    minVal = INF
    index = 0

    for i in range(1, n+1):
        if distance[i] < minVal and not visited[i]:
            minVal = distance[i]
            index = i

    return index

In [2]:
def dijkstra(start):
    distance[start] = 0
    visited[start] = True
    for j in graph[start]:
        distance[j[0]] = j[1]

    for i in range(n-1):
        # 현재 최단 거리가 가장 짧은 노드 방문 처리
        now = getSmallestNode()
        visited[now] = True

        # 현재 노드와 연결된 다른 노드 확인
        for j in graph[now]:
            coost = distance[now] + j[1]

            # 현재 노드를 거쳐서 다른 노드로 이동하는 거리가 더 짧은 경우
            if coost < distance[j[0]]:
                distance[j[0]] = coost

In [17]:
dijkstra(start)

for i in range(1, n+1):
    print(distance[i]) if distance[i] != INF else print("INFINITY")

0
2
3
1
2
4


### 개선된 다익스트라 알고리즘

시간복잡도 $O(ElogV)$ $(E: 간선의 개수, V: 노드의 개수)$를 보장하여 해결할 수 있음.

최단 거리가 가장 짧은 노드를 선형적으로 찾는 것이 아니라 힙 자료구조를 사용함. (-> `getSmallestNode()` 함수 필요 X)
즉, 현재 가장 가까운 노드를 저장하기 위한 목적으로만 우선순위 쿠를 추가로 이용함.

> 힙(Heap): Priority Queue를 구현하기 위해 사용함.
>> Max Heap: 값이 큰 데이터가 먼저 삭제됨. 부모가 제일 큰 완전 이진 트리.
>> Min Heap: 값이 작은 데이터가 먼저 삭제됨. 부모가 가장 작은 완전 이진 트리.

In [5]:
import heapq # heap 자료구조 사용

In [16]:
distance = [INF] * (n+1)

In [19]:
def betterDijkstra(start):
    # priority queue에 시작 노드 삽입
    q = []
    heapq.heappush(q, (0, start))
    distance[start] = 0

    while q: # queue가 비어있지 않다면
        dist, now = heapq.heappop(q) # 최단 거리가 짧은 노드에 대한 정보 꺼내기
        if distance[now] < dist: continue # 이미 처리 됐다면 무시

        # 현재 노드와 연결된 다른 노드 확인
        for i in graph[now]:
            coost = dist + i[1]

            # 현재 노드를 거쳐서, 다른 노드로 이동하는 거리가 더 짧은 경우
            if coost < distance[i[0]]:
                distance[i[0]] = coost
                heapq.heappush(q, (coost, i[0]))

In [20]:
betterDijkstra(start)

for i in range(1, n+1):
    print(distance[i]) if distance[i] != INF else print("INFINITY")

0
2
3
1
2
4


## 플로이드 워셜 알고리즘

Floyed-Warshall Algorithm

모든 지점에서 다른 모든 지점까지의 최단 경로를 모두 구해야 하는 경우 사용함.
DP로 구현함. 점화식을 다음과 같음.

$$D_{ab} = min(D_{ab}, D_{ak}+D_{kb}) $$

<br>

**과정**

1. 최단 거리를 가지는 노드를 하나씩 반복적으로 선택함.
2. 해당 노드를 거쳐 가는 경로를 확인함.
3. 최단 거리 테이블을 갱신함.

<br>

노드의 개수가 N개일 때, 알고리즘 상으로 N번의 단계를 수행하며, 각 단계마다 $O(N^2)$의 연산을 통해 현재노드를 거쳐가는 모든 경로를 고려함. 따라서 총 시간 복잡도는 $O(N^3)$임.

In [1]:
INF = int(1e9)

n = int(input())
m = int(input())
graph = [[INF] * (n+1) for _ in range(n+1)]

for i in range(1, n+1):
    for j in range(1, n+1):
        if i == j: graph[i][j] = 0

for _ in range(m):
    startNode, endNode, cost = map(int, input().split())
    graph[startNode][endNode] = cost

In [2]:
for k in range(1, n+1):
    for i in range(1, n+1):
        for j in range(1, n+1):
            graph[i][j] = min(graph[i][j], graph[i][k] + graph[k][j])

In [3]:
for i in range(1, n+1):
    for j in range(1, n+1):
        print(graph[i][j], end=" ") if graph[i][j] != INF else print("INFINITY", end=" ")
    print()

0 4 8 6 
3 0 7 9 
5 9 0 4 
7 11 2 0 
